## 데이터 및 라이브러리 로딩

In [1]:
import json
import re

In [34]:
## 전본

## 수정본

# 데이터 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\3. JSON 컨버팅\JSON byChapter\R078r3e_fullChapter_converted_ver1.json'

# 텍스트 추출
with open(data_path, 'rb') as source:
    dict_originJSON = json.load(source)

#print(''.join(lines))    

In [2]:
## 수정본

# 데이터 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트\R078r3am1e_processed.txt'

# 텍스트 추출
with open(data_path, 'rb') as source:
    lines_amendment = source.readlines()
    lines_amendment = [element.decode('utf-8') for element in lines_amendment]

#print(''.join(lines))    

## 전역 변수 정의

In [3]:
pattern_main_title = ("main", r'\s*\d{1,}\.\s+')
pattern_article = ("art", r'\s*(\d{1,}\.){2}\s+')
pattern_paragraph = ("para", r'\s*(\d{1,}\.){3}\s+')
pattern_sub_paragraph = ("paraS", r'\s*(\d{1,}\.){4}\s+')
pattern_item = ("item", r'\s*\([a-hj-u]\)\s+')
pattern_sub_item = ("itemS", r'\s*\([ivx]{1,}\)\s+')
pattern_description = ("dsc", r'\s*[A-Z]') # 추후에 수정이 필요할 수 있음

patternL = [pattern_main_title, pattern_article, pattern_paragraph, pattern_sub_paragraph, pattern_item, pattern_sub_item, pattern_description]
patternL_stage = [pattern_main_title, pattern_article, pattern_paragraph, pattern_sub_paragraph]

dict_tagName = {"item":"Item", "itemS":"Sub-item", "dsc":"Description",
                "main":"Chapter main title",
                "art":"Article", "para":"Paragraph", "paraS":"Sub-paragraph"}

tags_stage = ["main", "art", "para", "paraS"]

## 필요 함수 정의

### 테그 반환

In [5]:
def lineTagger(linesL_input, flag_return_onlyTag=True):
    
    ## 전역 변수 로딩
    global patternL
    
    ## 행 별로 패턴 일치 여부 검색 및 결과 테깅
    tupleL_tagInfo = []
    for tag, pattern in patternL:
        pattern = r"^" + fr"{pattern}"
        tupleL_tagInfo.append([(index, tag) for index, line in enumerate(linesL_input) if re.match(pattern, line)])
    
    ## 리스트 평탄화 및 오름차순 정렬
    tupleL_tagInfo = sum(tupleL_tagInfo,[])
    tupleL_tagInfo.sort()
    
    ## 예외처리 (조건 중복, item의 알파뱃 i ~ sub item의 로마자 i)
    pattern_ish = r'^\s*\(h\)\s'
    pattern_isi = r'^\s*\(i\)\s'
    pattern_isii = r'^\s*\(ii\)\s'

    #/ Check point 1: item (h)가 존재하는가? (문제가 될 수 있는 item (i)의 선행 조건)
    #/ True: check point 2 확인
    idxL_ish = [index for index, line in enumerate(linesL_input) if re.match(pattern_ish, line)]
    for idx_ish in idxL_ish:
   
        #// Check point 2: item (h)의 sub item이 존재하는가? (tagInfo만으로는 구분할 수 없는 요소) + 문단 종료에 따른 인덱싱 오류에 대해 대비할 필요가 있음
        #// True: check point 3-1 확인
        if re.match(pattern_isii, linesL_input[idx_ish+2]):
            idx_cursor = idx_ish + 3
            while (tupleL_tagInfo[idx_cursor][1]=="itemS"):
                idx_cursor += 1
            #/// Check point 3: 마지막 itemS로 테그된 행이 item (i)일 수도 있지 않은가?
            #/// True: 테그 변환    
            if re.match(pattern_isi, linesL_input[idx_cursor-1]):
                tupleL_tagInfo[idx_cursor-1] = (idx_cursor-1, "item")
   
    #// False: check point 3-2 확인
        else:
            idx_cursor = idx_ish + 1
            #/// Check point 3: item (h) 다음으로 itemS로 테그된 행이 item (i)이 존재하는가?
            #// True: 테그 변환
            if tupleL_tagInfo[idx_cursor][1]=="itemS":
                tupleL_tagInfo[idx_cursor] = (idx_cursor, "item")
    
    ## 결과 반환
    if flag_return_onlyTag:
        return([tag for idx, tag in tupleL_tagInfo])      
    else:
        return(tupleL_tagInfo)

### 인덱스 검색

In [6]:
def idxFinder(tags, idx_start, tag_target, tag_end=False):
    global tags_stage
    
    idxL_target = []
    idx_cursor = idx_start
    idx_end = len(tags) - 1
    #tags_out = tags_stage.copy().split()

    
    if tag_end:
        while (idx_cursor <= idx_end and tags[idx_cursor] not in tag_end):
            if tags[idx_cursor]==tag_target:
                idxL_target.append(idx_cursor)
            idx_cursor += 1
    
    else:
        while (idx_cursor <= idx_end):
            if tags[idx_cursor]==tag_target:
                idxL_target.append(idx_cursor)
            idx_cursor += 1
            
    return(idxL_target)

### 문장 페턴 반환

In [7]:
def patternFinder(target_tag, base_patternL):
    base_patternD = dict(base_patternL)
    target_pattern = base_patternD[target_tag]
    
    return(target_pattern)

### 입력 페턴을 바탕으로 문장 분해

In [9]:
def lineParser(line, break_pattern, sel_group, opt_strip=False):
    breaks = re.search(break_pattern, line)
    
    parsed_result = []   
    for idx in sel_group:
        if opt_strip:
            parsed_result.append(breaks.group(idx).strip())
        else:
            parsed_result.append(breaks.group(idx))
            
    return(parsed_result)

### 문항 번호에서 주소 정보 구체화

In [37]:
def addressParser(address):
    # Define the regex pattern
    pattern = r'(\d+)(?:\.(\d+))?(?:\.(\d+))?(?:\.(\d+))?.*'
    
    # Match the pattern to the sentence
    match = re.match(pattern, address)
    
    if match:
        # Extract the matched groups
        chapter = match.group(1)
        article = match.group(2)
        paragraph = match.group(3)
        sub_paragraph = match.group(4)
        
        # Create the result dictionary
        result = {
            'Chapter': chapter,
            'Article': article,
            'Paragraph': paragraph,
            'Sub-paragraph': sub_paragraph,
        }
        
        # Filter out None values
        result = {k: v for k, v in result.items() if v is not None}
        
        return result
    else:
        return None


'\n# Example usage\nsentences = [\n    "5.13.4.3 This particle should be (...)",\n    "7.21 This is another example (...)",\n    "8.9.2 Here is yet another example (...)"\n]\n\nfor sentence in sentences:\n    parsed_info = parse_structure(sentence)\n    if parsed_info:\n        print(parsed_info)\n    else:\n        print(f"Could not parse: {sentence}")\n'

In [ ]:
''' ## Example usage ##

sentences = [
    "5.13.4.3 This particle should be (...)",
    "7.21 This is another example (...)",
    "8.9.2 Here is yet another example (...)"
]

for sentence in sentences:
    parsed_info = parse_structure(sentence)
    if parsed_info:
        print(parsed_info)
    else:
        print(f"Could not parse: {sentence}")
'''

### JSON 구성 요소 생성 및 입력

In [10]:
## 독립형 sublistLoader (미사용)

def sublistLoader(lines, tags, base_dict, target_tag, pivot_idx, opt_parse=False, opt_strip=False):
    global patterL
    global dict_tagName
    global tags_stage

    tag_pivot = tags[pivot_idx]
    tags_end = tags_stage.copy()
    tags_end.append(tag_pivot)

    idxL_loader = idxFinder(tags, pivot_idx+1, target_tag, tag_end=tags_end)
    ##        
    lines_loader = []
    for idx_loader in idxL_loader:
        lines_loader.append(lines[idx_loader].strip())

    ##
    if len(lines_loader):
        if opt_parse:
            pattern_pivot = patternFinder(tag_pivot, patterL)
            ele_searchingKey = lineParser(lines[pivot_idx], rf"({pattern_pivot})" + r"(.*)", sel_group=[3], opt_strip=opt_strip)[0]
        else:
            ele_searchingKey = lines[pivot_idx].strip()
            
        idx_pivotInlist = base_dict[fullName_pivot].index(ele_searchingKey)

        fullName_pivot = dict_tagName[tag_pivot]
        base_dict[fullName_pivot][idx_pivotInlist].append(lines_loader)

In [11]:
def listLoader(lines, tags, base_dict, target_tag, pivot_idx, target_tag_subList=None):
    global dict_tagName
    global tags_stage
    
    idxL_loader = idxFinder(tags, pivot_idx+1, target_tag, tag_end=tags_stage)
    if target_tag_subList:
        idx_end = len(lines)-1
        idxL_with_subList = [idx_loader for idx_loader in idxL_loader if idx_loader < idx_end and tags[idx_loader+1]==target_tag_subList]
    ##        
    lines_loader = []
    for idx_loader in idxL_loader:
        if target_tag_subList and idx_loader in idxL_with_subList:
            lines_subLoader = []
            idxL_subLoader = idxFinder(tags, idx_loader+1, target_tag_subList, tag_end=target_tag)
            for idx_subLoader in idxL_subLoader: lines_subLoader.append(lines[idx_subLoader].strip())
            
            lines_loader.append([lines[idx_loader].strip(), lines_subLoader])
            
        else:     
            lines_loader.append(lines[idx_loader].strip())

    ##
    if len(lines_loader):
        fullName = dict_tagName[target_tag]
        base_dict[fullName] = lines_loader
        
## 재귀 함수를 이용한 구조로 변경할 수 있지 않을까? depth와 target tags를 이용한 확장성 확보

In [12]:
def dictLoader(lines, tags, base_dict, pivot_pattern, pivot_idx, opt_dsc=False, opt_item=False, opt_itemS=False):
    
    numbering, dsc_0 = lineParser(lines[pivot_idx], rf"({pivot_pattern})" + r"(.*)", sel_group=[1,3], opt_strip=True)
    dict_loaded = base_dict[numbering] = {}
    
    if opt_dsc:
        listLoader(lines, tags, dict_loaded, target_tag="dsc", pivot_idx=pivot_idx)
        if "Description" in list(dict_loaded.keys()): dict_loaded["Description"].insert(0,dsc_0)
        else: dict_loaded["Description"] = [dsc_0]
        
    if opt_item:
        if opt_itemS:
            listLoader(lines, tags, dict_loaded, target_tag="item", target_tag_subList="itemS", pivot_idx=pivot_idx)
        else:
            listLoader(lines, tags, dict_loaded, target_tag="item", pivot_idx=pivot_idx)
        
    return(dict_loaded)

## dsc만 예외 처리로 남겨 두고 나머지는 구체적인 변수로 받아서 반복 처리 (sub list에 대해서는 [..., [...]] 같은 구조로 처리할 수 있을 듯하다.)

In [13]:
def recursive_dictLoader(lines, tags, base_dict, init_idx, parental_tag):
    global patternL_stage
    tags_stage = [tag for tag, pattern in patternL_stage]
    
    idxL_parentalTag = idxFinder(tags=tags, idx_start=init_idx, tag_target=parental_tag, tag_end=tags_stage[:tags_stage.index(parental_tag)])
    pattern_parentalTag = patternFinder(parental_tag, patternL)
    
    for idx_parentalTag in idxL_parentalTag:
        
        dict_parental = dictLoader(lines, tags, base_dict, pattern_parentalTag, idx_parentalTag, opt_dsc=True, opt_item=True, opt_itemS=True)
        
        if parental_tag != tags_stage[-1]:
            idx_parentalStage = tags_stage.index(parental_tag)
            tag_child = tags_stage[idx_parentalStage+1]
            recursive_dictLoader(lines, tags, dict_parental, idx_parentalTag+1, tag_child)

In [14]:
def dictInitiator_chapter(lines, tags, main_line_idx):
    dict_return = {}
    line_main = lines_chapter[main_line_idx]
    num_chapter, title_chapter = [elm.strip() for elm in line_main.split('.')]
    dict_return["Chapter"] = num_chapter
    dict_return["Title"] = title_chapter
    
    listLoader(lines, tags, dict_return, target_tag="dsc", pivot_idx=main_line_idx)
    listLoader(lines, tags, dict_return, target_tag="item", target_tag_subList="itemS", pivot_idx=main_line_idx)
    
    return dict_return

## 워킹 스페이스

### 단락 구분 및 유형 확인

In [ ]:
# 단락 구분

 ## 단락 시작 지점 확인
patten_startSect = r".*?\bto read:"
idxs_startSect = [index for index, line in enumerate(lines) if re.match(patten_startSect, line)]

idxs_endSect = idxs_startSect.copy()
idxs_endSect.pop(0)
idxs_endSect.append(len(lines))

 ## 단락 구분 및 개별 저장
linesL_sect = []
for idx_startSect, idx_endSect in zip(idxs_startSect, idxs_endSect):
   lines_sect = lines[idx_startSect:idx_endSect]
   linesL_sect.append(lines_sect)

 ## 유형 확인 후 dict 자료형으로 전환
dictL_sect = []
for lines_sect in linesL_sect:
    line_startSect = lines_sect[0].lower()
    flag_type = ""
    
    if "amend" in line_startSect:
        flag_type = "Amend"
       
    elif "insert" in line_startSect:
        flag_type = "Insert"
        
    elif "renumber" in line_startSect:
        flag_type = "Renumber" 
        
    dict_sect = {"Type":flag_type, "Header":lines_sect[0], "Base":lines_sect[1:], "Tag":lineTagger(lines_sect[1:])}
    dictL_sect.append(dict_sect)

### JSON 변환

In [19]:
for dict_sect in dictL_sect:

    tags_stage = [tag for tag, pattern in patternL_stage]
    topStage = tags_stage[max(tags_stage.index(tag_inMainTarget) for tag_inMainTarget in dict_sect["Tag"] if tag_inMainTarget in tags_stage)]

    if topStage == "main":
        dict_baseToJSON= dictInitiator_chapter(dict_sect['Base'], dict_sect['Tag'], main_line_idx=0)
    else:
        dict_baseToJSON = {}

    #
    recursive_dictLoader(dict_sect['Base'], dict_sect['Tag'], dict_baseToJSON, init_idx=0, parental_tag=topStage)

    #
    dict_sect['JSON'] = dict_baseToJSON

### 주소 특정 및 수정 사항 적용

In [ ]:
for dict_sect in dictL_sect:

  for address_key in list(dict_sect["JSON"].keys()):
      
      address_info = list[addressParser(address_key).values()]
      address_depth = len(address_info)
      
      addressL = []
      tmp = ""
      for frag in address_info:
        tmp = tmp + frag + "."
        addressL.append(tmp)
      
      target_dict = dict_originJSON  
      for address in addressL:
        target_dict = target_dict[address]
        
      target_dict = dict_sect["JSON"][address_key]

## JSON 직렬화 처리 후 파일 저장

### 문서 전체

In [106]:
with open("R078r3e_fullchpter_converted_ver1.json", "w") as file:
    json.dump(dict_full, file)

### 개별 챕터

In [118]:
keys_chapter = list(filter(lambda x: "chapter" in x, list(dict_full.keys())))

for number_of_chapter in range(len(linesL_chapter)):
    with open(f"R078r3e_chapter{number_of_chapter+1}_converted_ver1.json", "w") as file:
        json.dump(dict_full[keys_chapter[number_of_chapter]], file)